In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import re
import json
import urllib
import urllib2
import StringIO
import datetime
import pandas as pd
import os
import numpy as np
pd.set_option('display.max_colwidth', 5000)

HOST = 'http://mtlog01-01-1t.yandex.ru:8123/'
import requests

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

### Версии Android

In [5]:
q = '''
    SELECT
        sum(Sign) as visits,
        dictGetString('OS', 'value', toUInt64(OS)) as os
    FROM visits_all
    WHERE StartDate = today()
        AND OSToRoot(OS) = 108
    GROUP BY os
    ORDER BY visits DESC
    LIMIT 5
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,os
0,1386305,Google Android 6.0 Marshmallow
1,1055798,Google Android 5.1 Lollipop
2,866112,Google Android 7.0 Nougat
3,802067,Google Android 4.4 KitKat
4,374637,Google Android 5.0 Lollipop


### Поиск -> Картинки

In [11]:
q = '''
    SELECT
        sum(Sign) as visits,
        SearchEngineID as search
    FROM visits_all
    WHERE StartDate = today()
        AND domain(Referer) = 'yandex.ru'
        AND extractURLParameter(Referer, 'from') LIKE 'yandex.ru;images%'
    GROUP BY search
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,search
0,1,0
1,16276,1
2,650,2


In [14]:
q = '''
    SELECT
        sum(Sign) as visits,
        extractURLParameter(Referer, 'from') as from
    FROM visits_all
    WHERE StartDate = today()
        AND domain(Referer) = 'yandex.ru'
        AND extractURLParameter(Referer, 'from') LIKE 'yandex.ru;images%'
        AND SearchEngineID = 2
    GROUP BY from
    ORDER BY visits DESC
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,from
0,676,yandex.ru;images%2Fpad%2Fsearch;images;;


### Рефереры навигационного suggest'a

In [17]:
q = '''
    SELECT
        sum(Sign) as visits,
        SearchEngineID as search
    FROM visits_all
    WHERE StartDate = today()
        AND domain(Referer) = 'yandex.ru'
        AND extractURLParameter(Referer, 'from') LIKE '%samsung-suggest-sdk-touch%'
    GROUP BY search
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,search


### Яндекс Работа

In [19]:
q = '''
    SELECT 
        sum(Sign) AS visits, 
        domain(Referer) AS ref_domain,
        SearchEngineID as search
    FROM visits_all 
    WHERE (StartDate = today()) AND (ref_domain LIKE '%rabota.yandex%')
    GROUP BY ref_domain, search
    ORDER BY visits DESC
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,ref_domain,search
0,1754,rabota.yandex.ru,0
1,390,rabota.yandex.ru,124
2,144,m.rabota.yandex.ru,124
3,70,rabota.yandex.by,0
4,33,rabota.yandex.ua,0
5,22,rabota.yandex.by,124
6,21,rabota.yandex.kz,0
7,5,m.rabota.yandex.by,124
8,2,rabota.yandex.kz,124
9,1,rabota.yandex.ua,124


### Android-app LinkedIn'a

In [21]:
q = '''
    SELECT 
        sum(Sign) AS visits, 
        TraficSourceID
    FROM visits_all 
    WHERE (StartDate = today()) AND (Referer = 'android-app://com.linkedin.android')
    GROUP BY TraficSourceID
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,TraficSourceID
0,65,8


### Новые браузеры Otter и ElementsBrowser

In [22]:
q = '''
    SELECT 
        sum(Sign) as visits,
        dictGetString('UserAgent', 'value', toUInt64(UserAgent)) as browser
    FROM visits_all
    WHERE StartDate = today()
        AND UserAgent IN (194, 195)
    GROUP BY browser
    FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,visits,browser
